In [3]:
from eval_1022 import *
import json
import numpy as np

qids = [
    "d5c745d00dba11ecb1e81171463288e9",
    "d5c0f39c0dba11ecb1e81171463288e9",
    "d5c164b20dba11ecb1e81171463288e9",
    "d5cbb1880dba11ecb1e81171463288e9",
    "d5c747c40dba11ecb1e81171463288e9",
    "d5c312260dba11ecb1e81171463288e9",
    "d5c3fdbc0dba11ecb1e81171463288e9",
    "d5c93d400dba11ecb1e81171463288e9",
    "d5c506940dba11ecb1e81171463288e9",
    "d5cacb920dba11ecb1e81171463288e9",
    "d5ca118e0dba11ecb1e81171463288e9",
    "d5c2f8e00dba11ecb1e81171463288e9",
    "d5c944fc0dba11ecb1e81171463288e9"
]


exp_name = "webqa_captions_counterfactual"
data_dir = "/home/pcarragh/dev/webqa/MultiModalQA/data/"
data_dir += "WebQA_train_val_obj_v2.json"

with open(data_dir, 'r') as f:
    eval_data = json.load(f)
# filter by qids
eval_data = {k: v for k, v in eval_data.items() if k in qids}

In [4]:
two_image_questions = {k:eval_data[k]['Q'] for k in tqdm(list(eval_data.keys())) if len(eval_data[k]["img_posFacts"]) == 2}
for k in two_image_questions:
    if not 'both' in two_image_questions[k]:
        print(k, two_image_questions[k])


  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:00<00:00, 104857.60it/s]

d5c0f39c0dba11ecb1e81171463288e9 "Is the Sumatran orangutan's skin on it's face as smooth as the skin on the Theropithecus gelada's face?"
d5c312260dba11ecb1e81171463288e9 "Is there a large tower that lights up at night in Las Vegas?"
d5c3fdbc0dba11ecb1e81171463288e9 "Do the soccer balls used in the FIFA World Cup always have the same pattern?"
d5c506940dba11ecb1e81171463288e9 "Is the bridge in the "Italian Landscape" painting by Jean-Victor Bertin larger than the bridge in the background of the painting "The Opening of Waterloo Bridge"?"
d5c944fc0dba11ecb1e81171463288e9 "Does SANAE_IV have more large buildings than Neumayer Station?"
d5ca118e0dba11ecb1e81171463288e9 "Are people restricted from touching the altar in the Church of the Holy Sepulchre in Jerusalem?"


In [5]:
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model, eval
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init

disable_torch_init()
model_path = "liuhaotian/llava-v1.5-7b"
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path, None, model_name
)

def llava_eval_on_webqa_sample(question, image_files):
    args = type('Args', (), {
        "model_path": model_path,
        "model_base": None,
        "model_name": model_name,
        "query": question,
        "conv_mode": None,
        "image_file": image_files,
        "sep": ",",
        "temperature": 0,
        "top_p": None,
        "num_beams": 1,
        "max_new_tokens": 512,
    })()

    return eval(tokenizer, model, image_processor,  args)

def webqa_accuracy(answer, label, Qcate):
    if Qcate == 'color':
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", color_set)
    elif Qcate == 'shape': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", shape_set)
    elif Qcate == 'yesno': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", yesno_set)
    elif Qcate == 'number': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", {"NUMBER"})
    else:
        return None
    return (F1_avg, F1_max, EM, RE_avg, PR_avg)

def accuracy_agg_results(qa_results):
    single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
    two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

    single_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in single_image_keys])
    two_image_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in two_image_keys])
    avr_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items()])
    return (single_acc, two_image_acc, avr_acc)

def accuracy_agg_generated_results(qa_results):
    single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
    two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

    single_acc = np.mean([PR_avg for key, dict in qa_results.items() if key in single_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()])
    two_image_acc = np.mean([PR_avg for key, dict in qa_results.items() if key in two_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()])
    avr_acc = np.mean([PR_avg for key, dict in qa_results.items() for idx, (_,_,_,_,PR_avg) in dict.items()])
    
    return (single_acc, two_image_acc, avr_acc)

def get_prompt(key, reverse_images = False):
    imgs = eval_data[key]['img_posFacts']
    if len(imgs) == 1:
        return f"<image-placeholder> \n Caption: {imgs[0]['title']} \n Question: {eval_data[key]['Q']}"
    assert(len(imgs) == 2)
    if reverse_images:
        return f"<image-placeholder> \n Caption: {imgs[1]['title']} \n <image-placeholder> \n Caption: {imgs[0]['title']} \n Question: {eval_data[key]['Q']}"
    return f"<image-placeholder> \n Caption: {imgs[0]['title']} \n <image-placeholder> \n Caption: {imgs[1]['title']} \n Question: {eval_data[key]['Q']}"



/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-22 17:24:15,091] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00

### LLAVA baseline on original images that have perturbations

In [12]:
from tqdm import tqdm

llava_results_baseline = {}
for k in list(eval_data.keys()):
    question = get_prompt(k)
    # print(question)
    image_files = ','.join([str(img_data['image_id']) for img_data in eval_data[k]['img_posFacts']])
    # try:
    answer = llava_eval_on_webqa_sample(question, image_files)
    # except:
    #     answer = 'ERROR: llava failed'
    label = eval_data[k]['A']
    eval_data[k]['A_llava'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_baseline[k] = webqa_accuracy(answer, label, Qcate)
    
    print(f"Question: {eval_data[k]['Q']}, Answer: {answer}, Label: {label}")

print(accuracy_agg_results(llava_results_baseline))

Question: "Is the Sumatran orangutan's skin on it's face as smooth as the skin on the Theropithecus gelada's face?", Answer: No, Label: ['"The Sumatran orangutan\'s skin on its face is not as as smooth as the skin on the Theropithecus gelada\'s face."']
Question: "is the woman holding something made of paper in both Sleeping Girl and The Love Letter?", Answer: Yes, the woman in both paintings is holding a piece of paper., Label: ['"Yes the woman is holding something made of paper in both Sleeping Girl and The Love Letter."']
Question: "Are all of the flags near the Broken track on Expedition Everest the same color?", Answer: No, the flags near the broken track on Expedition Everest are not all the same color. They come in various colors, adding to the visual appeal of the scene., Label: ['"All of the flags near the Broken track on Expedition Everest are not the same color."']
Question: "Is there a large tower that lights up at night in Las Vegas?", Answer: Yes, there is a large tower t

### Correct despite blank image

In [13]:
llava_results_blank = {}
for k in list(eval_data.keys()):
    question = get_prompt(k)
    image_files ='playground/counterfactual_exp/BLANK.jpg,playground/counterfactual_exp/BLANK.jpg'
    try:
        answer = llava_eval_on_webqa_sample(question, image_files)
    except:
        answer = 'ERROR: llava failed'
    label = eval_data[k]['A']
    eval_data[k]['A_llava_blank'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_blank[k] = webqa_accuracy(answer, label, Qcate)
    print(f"Question: {eval_data[k]['Q']}, Answer: {answer}, Label: {label}")
    
print(accuracy_agg_results(llava_results_blank))

Question: "Is the Sumatran orangutan's skin on it's face as smooth as the skin on the Theropithecus gelada's face?", Answer: No, Label: ['"The Sumatran orangutan\'s skin on its face is not as as smooth as the skin on the Theropithecus gelada\'s face."']
Question: "is the woman holding something made of paper in both Sleeping Girl and The Love Letter?", Answer: Yes, in both Sleeping Girl and The Love Letter, the woman is holding a piece of paper., Label: ['"Yes the woman is holding something made of paper in both Sleeping Girl and The Love Letter."']
Question: "Are all of the flags near the Broken track on Expedition Everest the same color?", Answer: Yes, Label: ['"All of the flags near the Broken track on Expedition Everest are not the same color."']
Question: "Is there a large tower that lights up at night in Las Vegas?", Answer: Yes, Label: ['"Yes, there is a large tower that lights up at night in Las Vegas."']
Question: "Do the soccer balls used in the FIFA World Cup always have the

### Correct despite perturbation

In [17]:
llava_results_perturbed_original_label = {}
llava_results_perturbed_generated_label = {}

perturbation_path = "/home/pcarragh/dev/webqa/image_gen_val/val_images_perturbed_gpt_obj_lama"

for k in list(eval_data.keys()):
    eval_data[k]['A_perturbed_llava'] = {}
    question = get_prompt(k)
    image_id = eval_data[k]['img_posFacts'][0]['image_id']
    
    # image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
    image_files = ""
    for img in eval_data[k]['img_posFacts']:
        generated_path = f"{perturbation_path}/{str(image_id)}_{k}.jpeg"
        if os.path.exists(generated_path):
            image_files += "," + generated_path
        else:
            image_files += "," + str(img['image_id'])
    try:
        answer = llava_eval_on_webqa_sample("Answer the following question based on the provided images. \n" + question, image_files[1:])
    except:
        answer = 'ERROR: llava failed'

    original_label = eval_data[k]['A']
    eval_data[k]['A_perturbed_llava'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    print(f"Question: {eval_data[k]['Q']}, Answer: {answer}, Label: {label}")
    llava_results_perturbed_original_label[k] = webqa_accuracy(answer, original_label, Qcate)

print(accuracy_agg_results(llava_results_perturbed_original_label))
print(accuracy_agg_results(llava_results_perturbed_generated_label))

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: "Is the Sumatran orangutan's skin on it's face as smooth as the skin on the Theropithecus gelada's face?", Answer: No, the Sumatran orangutan's skin on its face is not as smooth as the Theropithecus gelada's face. The Theropithecus gelada has a smoother, more hairless face, while the Sumatran orangutan has a more hairy and rougher face., Label: ['"The top side of the Leccinum Scabrum mushroom\'s cap is not the same color as the stem."']
Question: "is the woman holding something made of paper in both Sleeping Girl and The Love Letter?", Answer: Yes, in both paintings, the woman is holding a piece of paper in her hand., Label: ['"The top side of the Leccinum Scabrum mushroom\'s cap is not the same color as the stem."']
Question: "Are all of the flags near the Broken track on Expedition Everest the same color?", Answer: No, not all of the flags near the Broken track on Expedition Everest are the same color., Label: ['"The top side of the Leccinum Scabrum mushroom\'s cap is not t

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
llava_results_perturbed_original_label_reversed = {}
llava_results_perturbed_generated_label_reversed = {}

for k in tqdm(list(eval_data.keys())):
    llava_results_perturbed_original_label_reversed[k] = {}
    llava_results_perturbed_generated_label_reversed[k] = {}
    eval_data[k]['A_perturbed_llava_reversed'] = {}
    question = get_prompt(k, True)
    for idx, label in eval_data[k]['A_perturbed'].items():
        image_id = eval_data[k]['img_posFacts'][0]['image_id']
        image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
        if len(eval_data[k]['img_posFacts']) > 1:
            image_2 = eval_data[k]['img_posFacts'][1]['image_id']
            image_files = str(image_2) + ',' + image_files
        try:
            answer = llava_eval_on_webqa_sample(question, image_files)
        except:
            answer = 'ERROR: llava failed'

        original_label = eval_data[k]['A']
        eval_data[k]['A_perturbed_llava_reversed'][idx] = answer
        Qcate = eval_data[k]['Qcate'].lower()
        llava_results_perturbed_original_label_reversed[k][idx] = webqa_accuracy(answer, original_label, Qcate)
        llava_results_perturbed_generated_label_reversed[k][idx] = webqa_accuracy(answer, [label], Qcate)

print(accuracy_agg_generated_results(llava_results_perturbed_original_label_reversed))
print(accuracy_agg_generated_results(llava_results_perturbed_generated_label_reversed))

  0%|          | 0/572 [00:00<?, ?it/s]

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 572/572 [10:54<00:00,  1.14s/it]

(0.12626262626262627, 0.35291164658634533, 0.17006208769887463)
(0.7344877344877345, 0.5276104417670683, 0.6945091191307723)


In [3]:
llava_results_perturbed_original_label_double_gen = {}
llava_results_perturbed_generated_label_double_gen = {}

for k in tqdm(list(eval_data.keys())):
    llava_results_perturbed_original_label_double_gen[k] = {}
    llava_results_perturbed_generated_label_double_gen[k] = {}
    eval_data[k]['A_perturbed_llava_reversed'] = {}
    question = get_prompt(k, True)
    for idx, label in eval_data[k]['A_perturbed'].items():
        image_id = eval_data[k]['img_posFacts'][0]['image_id']
        image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
        if len(eval_data[k]['img_posFacts']) > 1:
            # image_2 = eval_data[k]['img_posFacts'][1]['image_id']
            image_files = image_files + ',' + image_files
        else:
            continue
        try:
            answer = llava_eval_on_webqa_sample(question, image_files)
        except:
            answer = 'ERROR: llava failed'

        original_label = eval_data[k]['A']
        eval_data[k]['A_perturbed_llava_reversed'][idx] = answer
        Qcate = eval_data[k]['Qcate'].lower()
        llava_results_perturbed_original_label_double_gen[k][idx] = webqa_accuracy(answer, original_label, Qcate)
        llava_results_perturbed_generated_label_double_gen[k][idx] = webqa_accuracy(answer, [label], Qcate)

print(accuracy_agg_generated_results(llava_results_perturbed_original_label_double_gen))
print(accuracy_agg_generated_results(llava_results_perturbed_generated_label_double_gen))

  0%|          | 0/572 [00:00<?, ?it/s]

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 572/572 [02:55<00:00,  3.26it/s]

(nan, 0.17369477911646586, 0.17369477911646586)
(nan, 0.6912650602409639, 0.6912650602409639)



/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
llava_results_perturbed_original_label_double_orig = {}
llava_results_perturbed_generated_label_double_orig = {}

for k in tqdm(list(eval_data.keys())):
    llava_results_perturbed_original_label_double_orig[k] = {}
    llava_results_perturbed_generated_label_double_orig[k] = {}
    eval_data[k]['A_perturbed_llava_reversed'] = {}
    question = get_prompt(k, True)
    for idx, label in eval_data[k]['A_perturbed'].items():
        image_id = eval_data[k]['img_posFacts'][0]['image_id']
        image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
        if len(eval_data[k]['img_posFacts']) > 1:
            image_2 = str(eval_data[k]['img_posFacts'][1]['image_id'])
            image_files = image_2 + ',' + image_2
        else: 
            continue
        try:
            answer = llava_eval_on_webqa_sample(question, image_files)
        except:
            answer = 'ERROR: llava failed'

        original_label = eval_data[k]['A']
        eval_data[k]['A_perturbed_llava_reversed'][idx] = answer
        Qcate = eval_data[k]['Qcate'].lower()
        llava_results_perturbed_original_label_double_orig[k][idx] = webqa_accuracy(answer, original_label, Qcate)
        llava_results_perturbed_generated_label_double_orig[k][idx] = webqa_accuracy(answer, [label], Qcate)

print(accuracy_agg_generated_results(llava_results_perturbed_original_label_double_orig))
print(accuracy_agg_generated_results(llava_results_perturbed_generated_label_double_orig))

  0%|          | 0/572 [00:00<?, ?it/s]

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 572/572 [02:49<00:00,  3.37it/s]

(nan, 0.7479919678714859, 0.7479919678714859)
(nan, 0.05823293172690764, 0.05823293172690764)



/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


: 

In [14]:
import pandas as pd

baseline_accs = accuracy_agg_results(llava_results_baseline)
blank_accs = accuracy_agg_results(llava_results_blank)
perturbed_original_label_accs = accuracy_agg_generated_results(llava_results_perturbed_original_label)
perturbed_generated_label_acc = accuracy_agg_generated_results(llava_results_perturbed_generated_label)

columns = ['experiment name', 'single_image', 'two_image', 'average']
accuracy_agg_df = pd.DataFrame(columns=columns)
accuracy_agg_df['experiment name'] = ['baseline', 'blank', 'perturbed_original_label', 'perturbed_generated_label']
accuracy_agg_df['single_image'] = [baseline_accs[0], blank_accs[0], perturbed_original_label_accs[0], perturbed_generated_label_acc[0]]
accuracy_agg_df['two_image'] = [baseline_accs[1], blank_accs[1], perturbed_original_label_accs[1], perturbed_generated_label_acc[1]]
accuracy_agg_df['average'] = [baseline_accs[2], blank_accs[2], perturbed_original_label_accs[2], perturbed_generated_label_acc[2]]
accuracy_agg_df.to_csv("results/{exp_name}.csv", index=False)
accuracy_agg_df


,experiment name,single_image,two_image,average
0,baseline,0.723977,0.729885,0.725175
1,blank,0.298977,0.000000,0.238345
2,perturbed_original_label,0.126263,0.000000,0.101863
3,perturbed_generated_label,0.734488,0.000000,0.592549


### Correct despite blank image and perturbation

In [15]:
# for questions that llava gets right regardless of having a blank image, check accuracy on counterfactual examples
perturbation_ignored = {key:idx for key, dict in llava_results_perturbed_original_label.items() for idx, (_,_,_,_,acc) in dict.items() if acc > 0}
blank_ignored = set([k for k, (_,_,_,_,acc) in llava_results_blank.items() if acc > 0])
parametric_ans_keys = set(perturbation_ignored.keys()).intersection(blank_ignored)
len(parametric_ans_keys)

60

In [16]:
for key in parametric_ans_keys:
    print(f"Q: {eval_data[key]['Q']}")
    print(f"A: {eval_data[key]['A']}")
    print(f"A_llava: {eval_data[key]['A_llava']}")
    print(f"A_llava_blank: {eval_data[key]['A_llava_blank']}")
    idx = perturbation_ignored[key]
    print(f"A_perturbed_llava: {eval_data[key]['A_perturbed_llava'][idx]}")
    
    print(f"{idx}, A_perturbed: {eval_data[key]['A_perturbed']}, A_perturbed_gpt: {eval_data[key]['A_perturbed_gpt']}")
    # , A_llava_blank: {eval_data[key]['A_llava_blank']}, A_perturbed_llava: {eval_data[key]['A_perturbed_llava']}")

Q: "What colors appeared on Dirk Nowitzki's uniform on October 9, 2009?"
A: ['"On October 9, 2009, Dirk Nowitzki\'s uniform was white, dark blue, light blue, and grey."']
A_llava: The colors on Dirk Nowitzki's uniform on October 9, 2009, were blue and white.
A_llava_blank: White
A_perturbed_llava: Blue and white
2, A_perturbed: {'1': 'black', '2': 'blue'}, A_perturbed_gpt: {'1': 'black', '2': 'blue'}
Q: "What color is the belly of a Green Tree Frog?"
A: ['"The belly of a Green Tree Frog is white."']
A_llava: The belly of a Green Tree Frog is white.
A_llava_blank: The belly of a Green Tree Frog is white.
A_perturbed_llava: The belly of a Green Tree Frog is white.
0, A_perturbed: {'0': 'pink'}, A_perturbed_gpt: {'0': 'pink'}
Q: "What are the colors of the Moscow torch?"
A: ['"The Moscow torch is red, white, and gold."']
A_llava: The Moscow torch is gold and white.
A_llava_blank: The Moscow torch is white and black.
A_perturbed_llava: The Moscow torch is gold and black.
3, A_perturbed: {'

: 